# comercio_ext_auxiliares.tb_auxiliar_vias
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/VIA_delta`
## 📌 Descrição do arquivo
Referência de **Vias de Transporte**.
|Coluna|Descrição|
|---|---|
|`CO_VIA`|Código da via|
|`NO_VIA`|Nome da via|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
def up(c): return F.upper(F.trim(F.col(c).cast(T.StringType())))

In [0]:
bronzePath  = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/via/"
silverTable = "silver_comercio_ext_auxiliares.tb_auxiliar_vias"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_VIA",   T.StringType(),   False),
    T.StructField("NO_VIA",   T.StringType(),   False),
    T.StructField("TS_REF",   T.TimestampType(),False),
    T.StructField("NM_ORIGEM",T.StringType(),   False),
])


## Extração
> #### **saprk.read**

In [0]:

df_bronze = spark.read.format("delta").load(bronzePath)



##Normalização
> #### **datatype**

In [0]:
df_norm = (
    df_bronze
    .withColumn("CO_VIA", up("CO_VIA"))
    .withColumn("NO_VIA", F.col("NO_VIA").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/via"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_VIA").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_VIA"])

In [0]:
df_silver = df_dedup.select("CO_VIA","NO_VIA","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, silverTable)
merge_condition = "t.CO_VIA = s.CO_VIA"

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "NO_VIA":   "s.NO_VIA",
        "TS_REF":   "s.TS_REF",
        "NM_ORIGEM":"s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
)